In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# Importación de librerias necesarias

In [ ]:
# 1. Importación de librerías necesarias
import pandas as pd
import numpy as np


# Carga del conjunto de datos 'Train'

In [ ]:

processed_path_train = '/content/drive/MyDrive/Colab Notebooks/Proyecto-TFG/data/train.csv'
train_df = pd.read_csv(processed_path_train)
print("Cargado conjunto de entrenamiento. Dimensiones:", train_df.shape)

# Lista de variables que queremos identificar


In [ ]:
variables = {
    "Child's age": "sc_age_years",
    "Sex": "sc_sex",
    "Mother's age": "a1_age",
    "Allergies": "allergies",
    "Arthritis": "autoimmune",
    "Ashtma": "k2q40a",
    "Brain injury": "concussion",
    "Headaches": "headache",
    "Anxiety": "anxiety",
    "Depression": "k2q32a",
    "Insurance": "currcov",
    "Alcohol": "ace9",
    "Race": ["sc_racer", "sc_race_r"],  # Dos posibles nombres
    "Family structure": "family_r",
    "Mother's education": "higrade",
    "Very LBW - LBW": "birthwt",
    "Poverty": "fpl_i1"
}

In [ ]:
# Verificar si las variables están en el dataset y mostrar valores únicos
for var_name, col in variables.items():
    if isinstance(col, list):  # Si hay más de un posible nombre (ej. "Race")
        found = False
        for c in col:
            if c in train_df.columns:
                found = True
                print(f"✅ '{var_name}' está en el dataset como '{c}'")
                print(f"   Valores únicos: {train_df[c].unique()}\n")
                break
        if not found:
            print(f"❌ '{var_name}' no está en el dataset\n")
    else:
        if col in train_df.columns:
            print(f"✅ '{var_name}' está en el dataset como '{col}'")
            print(f"   Valores únicos: {train_df[col].unique()}\n")
        else:
            print(f"❌ '{var_name}' no está en el dataset\n")

In [ ]:
variables_articulo = [
    "sc_age_years", "sc_sex", "a1_age", "allergies", "autoimmune", "k2q40a",
    "concussion", "headache", "k2q32a", "ace9",
    "sc_racer", "family_r", "higrade", "birthwt", "fpl_i1"
]

# Variables presentes en el dataset
variables_presentes = [var for var in variables_articulo if var in train_df.columns]

# Variables que faltan
variables_faltantes = [var for var in variables_articulo if var not in train_df.columns]

# Mostrar resultados
print("Variables presentes en el dataset:")
print(variables_presentes)

print("\nVariables faltantes en el dataset:")
print(variables_faltantes)


In [ ]:
#Visualizamos los datos
for col in train_df.columns:
    print(f"\nColumna: {col}")
    print(train_df[col].unique())


**Tratamiento rango de valores numéricos**

In [ ]:
import re
import numpy as np
import pandas as pd

def limpiar_y_convertir_a_int(train_df, columnas):
    """
    Limpia y convierte columnas con valores como '50 or more' o '40 or less' a enteros,
    manteniendo NaNs, usando el tipo pandas 'Int64'.
    """
    def extraer_entero(valor):
        if isinstance(valor, str):
            match = re.match(r"(\d+)", valor)
            if match:
                return int(match.group(1))
            else:
                return np.nan
        elif isinstance(valor, (int, float)) and not pd.isna(valor):
            return int(valor)
        else:
            return np.nan

    for col in columnas:
        if col in train_df.columns:
            print(f"→ Limpiando y convirtiendo '{col}'...")
            train_df[col] = train_df[col].apply(extraer_entero).astype("Int64")

    return train_df


In [ ]:
def convertir_cadenas_a_enteros(train_df, columnas):
    """
    Convierte valores como '2009' (strings numéricos) a enteros,
    dejando como NaN los que no se pueden convertir.
    """
    for col in columnas:
        if col in train_df.columns:
            print(f"→ Convirtiendo strings numéricos en '{col}' a enteros...")
            train_df[col] = pd.to_numeric(train_df[col], errors='coerce').astype('Int64')
    return train_df

# Corregimos tipos de datos numericos

In [ ]:
columnas_a_corregir = ['birth_yr', 'a1_age', 'fpl_i1']  # O cualquier otra columna con años como strings
train_df = convertir_cadenas_a_enteros(train_df, columnas_a_corregir)


In [ ]:
# Columnas que quieres limpiar y convertir a enteros
columnas_a_tratar = ['fpl_i1', 'a1_age', 'sc_age_years']  # Añade las que quieras

# Aplicar la limpieza
train_df = limpiar_y_convertir_a_int(train_df, columnas_a_tratar)


In [ ]:
for col in train_df.columns:
    print(f"\nColumna: {col}")
    print(train_df[col].unique())


# Estudio de valores nulos

In [ ]:
#Estudio de valores nulos
nulos_por_columna = train_df.isnull().sum()
no_nulos_por_columna = train_df.notnull().sum()

resumen_nulos = pd.DataFrame({
    'Columna': train_df.columns,
    'Valores Nulos': nulos_por_columna,
    'Valores No Nulos': no_nulos_por_columna
})


# Permitir que pandas muestre todas las filas
pd.set_option('display.max_rows', None)

# Mostrar el DataFrame con todas las filas
print(resumen_nulos)

In [ ]:
#Analizar la cantidad de valores nulos
# Calcular valores nulos por columna
missing_values = train_df.isnull().sum()

# Calcular porcentaje de valores nulos
missing_percentage = (missing_values / len(train_df)) * 100

# Crear un DataFrame resumen
missing_data = pd.DataFrame({'Valores Nulos': missing_values, '% de Nulos': missing_percentage})
missing_data = missing_data.sort_values(by='Valores Nulos', ascending=False)

print(missing_data)


# Eliminamos repetidos

In [ ]:
# Eliminamos columnas repetidas
train_df = train_df.drop(columns=['cerpals_desc.1'])
train_df = train_df.drop(columns=['k2q31c.1'])
train_df = train_df.drop(columns=['k2q40a.1'])

In [ ]:
for col in train_df.columns:
    print(f"\nColumna: {col}")
    print(train_df[col].unique())

In [ ]:
# Tamaño del DataFrame
print("📐 Shape del DataFrame (filas, columnas):")
print(train_df.shape)

# Número de filas y columnas por separado
print(f"\n📄 Número de filas: {train_df.shape[0]}")
print(f"📊 Número de columnas: {train_df.shape[1]}")

# Conteo de tipos de datos
print("\n📚 Tipos de datos en el DataFrame:")
print(train_df.dtypes.value_counts())

# Columnas que todavía tienen texto (tipo object)
object_cols = [col for col in train_df.columns if train_df[col].dtype == 'object']
print(f"\n🟨 Columnas tipo 'object' (texto): {len(object_cols)}")
print(object_cols)

# Columnas con valores nulos
columnas_con_nulos = train_df.columns[train_df.isnull().any()]
print(f"\n💧 Columnas con valores nulos: {len(columnas_con_nulos)}")
print(columnas_con_nulos.tolist())

# Total de valores nulos en el dataset
total_nulos = train_df.isnull().sum().sum()
print(f"\n🔢 Total de valores nulos en todo el dataset: {total_nulos}")

# Porcentaje de valores nulos por columna
print("\n📉 Porcentaje de valores nulos por columna (si hay):")
porcentaje_nulos = train_df.isnull().mean() * 100
print(porcentaje_nulos[porcentaje_nulos > 0].sort_values(ascending=False))

# Columnas con una sola categoría (no aportan al modelo)
columnas_sin_variabilidad = train_df.columns[train_df.nunique(dropna=True) <= 1]
print(f"\n⚠️ Columnas con solo una categoría: {len(columnas_sin_variabilidad)}")
print(columnas_sin_variabilidad.tolist())


In [ ]:
import numpy as np
import pandas as pd

# ========== Diccionario de tipos de variables ==========
variables_tipos = {
    "k7q84_r": "Ordinal", "hardwork": "Ordinal", "k8q34": "Ordinal", "k8q32": "Ordinal",
    "k8q31": "Ordinal", "wktosolve": "Ordinal", "talkabout": "Ordinal", "sharetoys": "Ordinal",
    "waitforturn": "Ordinal", "outdoorswkend": "Ordinal", "calmdown_r": "Ordinal", "temper_r": "Ordinal",
    "nameemotions": "Ordinal", "focuson": "Ordinal", "bullied_r": "Ordinal", "k7q70_r": "Ordinal",
    "k7q83_r": "Ordinal", "outdoorswkday": "Ordinal", "screentime": "Ordinal", "k7q85_r": "Ordinal",
    "sc_k2q22": "Binaria", "higrade": "Ordinal", "birth_yr_f": "Categórica", "birthwt": "Categórica",
    "birthwt_l": "Binaria", "family_r": "Categórica", "sc_racer": "Categórica", "agepos4": "Ordinal",
    "sc_english": "Ordinal", "hcability": "Ordinal", "grades": "Categórica", "makefriend": "Ordinal",
    "k3q04_r": "Categórica", "bedtime": "Ordinal", "k7q33": "Ordinal", "k8q30": "Ordinal",
    "k8q21": "Ordinal", "k7q82_r": "Ordinal", "fpl_i1": "Numérica", "k6q73_r": "Ordinal",
    "k6q70_r": "Ordinal", "a1_menthealth": "Ordinal", "a1_physhealth": "Ordinal", "k6q71_r": "Ordinal",
    "k7q04r_r": "Ordinal", "a1_relation": "Categórica", "a1_marital": "Categórica", "k2q33b": "Binaria",
    "k2q32b": "Binaria", "k2q34b": "Binaria", "k2q31b": "Binaria", "k2q31d": "Binaria", "addtreat": "Binaria",
    "confirminjury": "Binaria", "sescurrsvc": "Binaria", "cerpals_desc": "Ordinal", "k2q34c": "Ordinal",
    "k2q31c": "Ordinal", "hhlanguage": "Categórica", "birth_yr": "Categórica", "a1_age": "Numérica",
    "memorycond": "Binaria", "allergies": "Binaria", "k2q40a": "Binaria", "autoimmune": "Binaria",
    "headache": "Binaria", "k2q33a": "Binaria", "k2q32a": "Binaria", "k2q34a": "Binaria", "k2q31a": "Binaria",
    "concussion": "Binaria", "k4q23": "Binaria", "k6q15": "Binaria", "ace9": "Binaria", "a1_sex": "Categórica",
    "a1_born": "Categórica", "sc_age_years": "Ordinal", "sc_sex": "Categórica", "a1_grade": "Categórica"
}

# ========== Diccionario de mapeos ==========
mapeos = {}

# ========== Función de mapeo ==========
def generar_mapeo(columna, tipo):
    if tipo == "Binaria":
        return {
            "Yes": 1, "No": 0,
            "No valid response": np.nan, "Logical skip": -1, "Not in universe": np.nan
        }
    else:
        valores_unicos = columna.dropna().unique()
        mapeo = {valor: idx + 1 for idx, valor in enumerate(sorted(valores_unicos))}
        mapeo.update({"Logical skip": -1, "Not in universe": np.nan, "No valid response": np.nan})
        return mapeo

# ========== Mapear solo variables no numéricas ==========
def mapear_dataset(train_df, variables_tipos):
    for columna, tipo in variables_tipos.items():
        if columna in train_df.columns and tipo != "Numérica":
            print(f"Mapeando columna: {columna}")
            mapeo = generar_mapeo(train_df[columna], tipo)
            mapeos[columna] = mapeo
            train_df[columna] = train_df[columna].map(mapeo)
    return train_df

# ========== USO ==========
# train_df debe estar previamente cargado
train_df = mapear_dataset(train_df, variables_tipos)

# ========== Revisión ==========
print("\n✅ Dataset tras mapeo a valores numéricos:")
print(train_df.head())
print("\n📊 Tipos de datos después del mapeo:")
print(train_df.dtypes)


In [ ]:
for columna, mapa in mapeos.items():
    print(f"\n📌 Columna: {columna}")
    for clave, valor in mapa.items():
        print(f"  '{clave}' → {valor}")

In [ ]:
for col in train_df.columns:
    print(f"\nColumna: {col}")
    print(train_df[col].unique())

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

def knn_univariado_imputation(train_df, variables_tipos, mapeos, n_neighbors=5):
    """
    Imputación univariada KNN por columna. Cada columna se imputa por sus propios vecinos más cercanos.
    """
    df_imputado = train_df.copy()

    for col in df_imputado.columns:
        if df_imputado[col].isnull().sum() == 0:
            continue  # Saltar columnas sin nulos

        print(f"🔧 Imputando columna: {col}")
        tipo = variables_tipos.get(col, "Numérica")
        valores_col = df_imputado[col].values

        # Índices de valores nulos y válidos
        idx_nulos = np.where(pd.isna(valores_col))[0]
        idx_validos = np.where(~pd.isna(valores_col))[0]

        for idx in idx_nulos:
            vecinos = valores_col[idx_validos]

            # Si hay menos vecinos válidos que K, usar todos
            if len(vecinos) == 0:
                imputado = 0 if tipo == "Numérica" else -1  # fallback
            else:
                vecinos_mas_cercanos = vecinos[:n_neighbors]  # no hay orden real, usamos primeros K

                if tipo == "Numérica":
                    imputado = np.nanmean(vecinos_mas_cercanos)
                    # Si la columna original es Int64, redondeamos
                    if pd.api.types.is_integer_dtype(train_df[col].dtype):
                        imputado = int(round(imputado))
                else:
                    imputado = Counter(vecinos_mas_cercanos).most_common(1)[0][0]

            df_imputado.at[idx, col] = imputado

        # Validación con mapeo
        if tipo in ["Binaria", "Ordinal", "Categórica"] and col in mapeos:
            valores_validos = set(v for v in mapeos[col].values() if not pd.isna(v))
            df_imputado[col] = df_imputado[col].apply(
                lambda x: min(valores_validos, key=lambda v: abs(v - x)) if x not in valores_validos else x
            )
            df_imputado[col] = df_imputado[col].astype(int)

    print("✅ Imputación univariada KNN finalizada.")
    return df_imputado


In [ ]:
train_df = knn_univariado_imputation(train_df, variables_tipos, mapeos)

# Guardar el DataFrame limpio en Google Drive
output_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto-TFG/data/1_train_df_clean.csv"
train_df.to_csv(output_path, index=False)

print(f"✅ Dataset guardado en: {output_path}")

In [ ]:
clean_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto-TFG/data/1_train_df_clean.csv"
c_df = pd.read_csv(clean_path)

In [ ]:
#Volvemos a comprobar si hay nulos
nulos_por_columna = c_df.isnull().sum()
no_nulos_por_columna = c_df.notnull().sum()

resumen_nulos = pd.DataFrame({
    'Columna': c_df.columns,
    'Valores Nulos': nulos_por_columna,
    'Valores No Nulos': no_nulos_por_columna
})


# Permitir que pandas muestre todas las filas
pd.set_option('display.max_rows', None)

# Mostrar el DataFrame con todas las filas
print(resumen_nulos)

In [ ]:
for col in c_df.columns:
    print(f"\nColumna: {col}")
    print(train_df[col].unique())

In [ ]:
# Inspección general de todas las columnas en train_df
for col in c_df.columns:
    print(f"📌 Columna: {col}")
    print(f"🔸 Valores únicos: {c_df[col].unique()}")
    print(f"🔸 Cantidad de valores únicos: {c_df[col].nunique(dropna=True)}")
    print(f"🔸 Porcentaje de nulos: {c_df[col].isnull().mean() * 100:.2f}%")
    print("-" * 60)


**Desmapear para obtener valores originales limpios**

In [ ]:
def desmapear_dataset(c_df, mapeos, variables_tipos):
    """
    Reemplaza valores numéricos por sus valores originales según los mapeos invertidos.
    """
    df_desmapeado = c_df.copy()

    for col, tipo in variables_tipos.items():
        if col in df_desmapeado.columns and col in mapeos and tipo != "Numérica":
            # Crear mapeo inverso: valor_numérico -> valor_original
            mapeo_original = mapeos[col]
            mapeo_inverso = {v: k for k, v in mapeo_original.items() if pd.notna(v)}

            print(f"🔁 Desmapeando columna: {col}")
            df_desmapeado[col] = df_desmapeado[col].map(mapeo_inverso)

    return df_desmapeado



In [ ]:
c_df = desmapear_dataset(c_df, mapeos, variables_tipos)

In [ ]:
for col in c_df.columns:
    print(f"\nColumna: {col}")
    print(c_df[col].unique())


In [ ]:
#Volvemos a comprobar si hay nulos
nulos_por_columna = c_df.isnull().sum()
no_nulos_por_columna = c_df.notnull().sum()

resumen_nulos = pd.DataFrame({
    'Columna': c_df.columns,
    'Valores Nulos': nulos_por_columna,
    'Valores No Nulos': no_nulos_por_columna
})


# Permitir que pandas muestre todas las filas
pd.set_option('display.max_rows', None)

# Mostrar el DataFrame con todas las filas
print(resumen_nulos)

In [ ]:
# Guardar el DataFrame limpio y desmapeado en Google Drive
output_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto-TFG/data/2_train_df_clean.csv"
c_df.to_csv(output_path, index=False)

print(f"✅ Dataset guardado en: {output_path}")